In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import similarities
from gensim import corpora, models, similarities

# spacy for lemmatization
import spacy

import nltk
from nltk.corpus import stopwords 
import string
from nltk import word_tokenize

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# f = open("/Users/dhams/Desktop/Research/PreProcessing/plot-genie-index-stop-removal.txt","r")
# g = open("/Users/dhams/Desktop/Research/PreProcessing/clean-romance-pg.txt","r")
# h = open("/Users/dhams/Desktop/Research/PreProcessing/adventure-pg-clean.txt","r")
# i = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-argosy-1917.txt","r")
# j = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Dime-Detective.txt","r")
# k = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Popular-Detective.txt","r")
# l = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Science_Wonder_Stories.txt","r")
# m = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Terror_Tales.txt","r")
f = open("../pre-processing/combined/combined-everything.txt","r")

In [3]:
# lines_1 = f.readlines()
# lines_2 = g.readlines()
# lines_3 = h.readlines()
# lines_4 = i.readlines()
# lines_5 = j.readlines()
# lines_6 = k.readlines()
# lines_7 = l.readlines()
# lines_8 = m.readlines()
# f.close()
# g.close()
# h.close()
# i.close()
# j.close()
# k.close()
# l.close()
# m.close()
# lines = lines_4 + lines_5 + lines_7 + lines_8 + lines_1 + lines_2 + lines_3 + lines_6
lines = f.readlines()

In [4]:
# def send_tokens(line):
#     tokens = word_tokenize(line)
#     return tokens

In [5]:
# from nltk.stem.wordnet import WordNetLemmatizer

# def lemmatization(texts):
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_word_list = []
#     for line in texts:
#         tokens = send_tokens(line)
#         lemmatized_word_list.append([lemmatizer.lemmatize(w) for w in tokens])        
#     return lemmatized_word_list

In [6]:
# lemmatized_tokens = lemmatization(lines)

In [7]:
lemmatized_tokens = []
for line in lines:
    tokens = word_tokenize(line)
    lemmatized_tokens.append([w for w in tokens if len(w)!=1])

In [8]:
lemmatized_tokens

[['murder'],
 ['taken', 'list', 'second', 'list', 'third', 'list'],
 ['scientific', 'modern', 'detective', 'mystery', 'story'],
 ['politician'],
 ['countess'],
 ['statesman'],
 ['college', 'student'],
 ['dictator'],
 ['modiste'],
 ['clown'],
 ['ringmaster'],
 ['impresario'],
 ['motion', 'picture', 'director'],
 ['translator'],
 ['manikin'],
 ['philanthropist'],
 ['bon', 'vivant'],
 ['archaeologist'],
 ['noblewoman'],
 ['spiritualist'],
 ['trophy', 'hunter'],
 ['society', 'belle'],
 ['merchant'],
 ['captain'],
 ['broker'],
 ['probation', 'officer'],
 ['doctor'],
 ['manufacturer'],
 ['paramour'],
 ['healer'],
 ['woman', 'nurse'],
 ['juror'],
 ['palmist'],
 ['foreign', 'emissary'],
 ['engineer'],
 ['deacon'],
 ['governor'],
 ['popular', 'co-ed'],
 ['professor'],
 ['dramatic', 'critic'],
 ['actress'],
 ['artist'],
 ['gambler'],
 ['curio', 'collector'],
 ['woman', 'librarian'],
 ['princess'],
 ['debutante'],
 ['ship', 'owner'],
 ['reformer'],
 ['egyptologist'],
 ['model'],
 ['psychologist']

In [9]:
id2word = corpora.Dictionary(lemmatized_tokens)

# Create Corpus
texts = lemmatized_tokens

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
topics = lda_model.print_topics()
doc_lda = lda_model[corpus]

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=lemmatized_tokens, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.461132111748542

Coherence Score:  0.3746720375574567


In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.277399 -0.185328       1        1  21.621387
9      0.099675  0.001027       2        1  15.587655
3      0.168087  0.318912       3        1  14.897574
5      0.063107 -0.115218       4        1  14.749403
2     -0.031788  0.035098       5        1   9.680889
8     -0.110647 -0.018656       6        1   7.140265
4     -0.081292 -0.013568       7        1   6.247084
7     -0.120931 -0.007199       8        1   4.636879
1     -0.125712 -0.002165       9        1   3.504226
0     -0.137899 -0.012903      10        1   1.934636, topic_info=      Category           Freq                Term          Total  loglift  \
term                                                                        
9      Default  105354.000000                 the  105354.000000  30.0000   
5      Default  103440.000000                 and  103440.000000  29.0000   
446    Default   80617.000000                  is   80617.000000  28.0000   
349    Default   61519.000000                  of   61519.000000  27.0000   
416    Default   46305.000000                  in   46305.000000  26.0000   
496    Default   41127.000000                  to   41127.000000  25.0000   
698    Default   35955.000000                that   35955.000000  24.0000   
727    Default   33992.000000                  it   33992.000000  23.0000   
1033   Default   30230.000000               there   30230.000000  22.0000   
965    Default   18529.000000                 his   18529.000000  21.0000   
422    Default   18335.000000                with   18335.000000  20.0000   
674    Default   35242.000000                 not   35242.000000  19.0000   
1586   Default   15920.000000                  he   15920.000000  18.0000   
2244   Default   21693.000000                  no   21693.000000  17.0000   
679    Default   12512.000000                more   12512.000000  16.0000   
511    Default   10052.000000                 are   10052.000000  15.0000   
634    Default    9588.000000               which    9588.000000  14.0000   
433    Default   14400.000000                 The   14400.000000  13.0000   
3378   Default   11141.000000                  so   11141.000000  12.0000   
677    Default   12358.000000                  be   12358.000000  11.0000   
6655   Default   11366.000000                 you   11366.000000  10.0000   
2275   Default   13992.000000                 all   13992.000000   9.0000   
3934   Default    8792.000000                this    8792.000000   8.0000   
3487   Default    7086.000000              little    7086.000000   7.0000   
854    Default    6959.000000                than    6959.000000   6.0000   
967    Default    6114.000000                time    6114.000000   5.0000   
717    Default    8555.000000                 had    8555.000000   4.0000   
227    Default   10355.000000                  an   10355.000000   3.0000   
1031   Default    8364.000000                 for    8364.000000   2.0000   
676    Default    8822.000000                 any    8822.000000   1.0000   
...        ...            ...                 ...            ...      ...   
84807  Topic10      18.461746               lilac     252.733749   1.3286   
28751  Topic10      29.971489               liver     420.914124   1.3031   
3511   Topic10      34.737034            transfer     494.427521   1.2897   
70383  Topic10      22.911314              achair     318.124390   1.3144   
38324  Topic10      24.282856           comparing     341.750458   1.3009   
10991  Topic10     193.674286             Project    3817.943359   0.9640   
82457  Topic10     186.862549              Cranly    4100.965820   0.8566   
84270  Topic10     116.978256         Gutenbergtm    2322.587891   0.9568   
83091  Topic10      68.412468               Lynch    1233.193115   1.0534   
17081  Topic10      65.423843          electronic    1182.692627   1.0506  

In [9]:
lda_model =  models.LdaModel.load('lda_model.model')

In [10]:
s = open("../pre-processing/combined/combined-source.txt","r")
source_lines = s.readlines()
source_bow = [id2word.doc2bow(text.lower().split()) for text in source_lines]
source_lda = lda_model[source_bow]

In [11]:
index = similarities.MatrixSimilarity(source_lda.corpus)

In [12]:
sus_test = open("../pre-processing/combined/combined-test-suspected-algo.txt","r")
sus_test_lines = sus_test.readlines()
sus_test_bow = [id2word.doc2bow(text.lower().split()) for text in sus_test_lines]
sus_test_lda = lda_model[sus_test_bow]

In [ ]:
index.save("simIndex.index")

In [13]:
sims = index[sus_test_lda.corpus]
#sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [36]:
for j in range(0,len(sus_test_lines)):
    #print("Length:",len(sus_test_lines[j].split()))
    flag = 0
    if(len(sus_test_lines[j].split())>1):
        for i in range(0,len(sims[j])):
            if(len(lines[i].split())>1):
                if sims[j][i]>0.8:
                    if(flag==0):
                        print("\nSimilarity of sentence:",sus_test_lines[j])
                        flag = 1
                    print((i+1),".",lines[i],":",sims[j][i])


Similarity of sentence: plan steal building

14300 . fatal ambition promise cause loss citizenship 
 : 0.81649655

Similarity of sentence: judgment poor judgment

14443 . discredit damning legal evidence 
 : 0.8944272
14534 . secret service operator employed 
 : 0.8944272

Similarity of sentence: gentleman friend oursa friend

3752 . horseback swimming flooded river
 : 0.8164966
5498 . match opposed uncle aunt 
 : 0.8164966
5592 . beloved distraught threatened loss fortune 
 : 0.8164966
5816 . avaricious cranky energetic 
 : 0.8164966
12791 . confronted danger life wife 
 : 0.8164966
14451 . spy intercepted 
 : 0.8164966
14540 . enemy 's path obstructed embankment 
 : 0.8164966
15621 . faulty design 
 : 0.8164966
15648 . house mystery 
 : 0.8164966
15654 . treasure box 
 : 0.8164966
15809 . old castle 
 : 0.8164966
15815 . escaped maniac 
 : 0.8164966
16228 . substance cushion
 : 0.8164966

Similarity of sentence: dont mean

8966 . ending story apar ending famous french story sculptor

In [15]:
lda_model.save("lda_model.model")

In [34]:
non_sus_test = open("../pre-processing/combined/combined-test-non-suspected-algo.txt","r")
non_sus_test_lines = sus_test.readlines()
non_sus_test_bow = [id2word.doc2bow(text.lower().split()) for text in sus_test_lines]
non_sus_test_lda = lda_model[sus_test_bow]

In [35]:
sims_non = index[non_sus_test_lda.corpus]

In [ ]:
for j in range(0,len(non_sus_test_lines)):
    #print("Length:",len(sus_test_lines[j].split()))
    flag = 0
    if(len(sus_test_lines[j].split())>1):
        for i in range(0,len(sims[j])):
            if(len(lines[i].split())>1):
                if sims[j][i]>0.7:
                    if(flag==0):
                        print("\nSimilarity of sentence:",sus_test_lines[j])
                        flag = 1
                    print((i+1),".",lines[i],":",sims[j][i])